In [1]:
import math, torch
from torch import nn
from d2l import torch as d2l

In [12]:
#@save
def masked_softmax(X, valid_lens):
    """通过在最后一个轴上掩蔽元素来执行softmax操作"""
    # X:3D张量，valid_lens:1D或2D张量
    if valid_lens is None:
        return nn.functional.softmax(X, dim=-1)
    else:
        shape = X.shape     # [2, 2, 4]
        if valid_lens.dim() == 1:
            print(f'valid_lens:{valid_lens}\n{valid_lens.shape}')
            valid_lens = torch.repeat_interleave(valid_lens, shape[1])
            print(f'valid_lens:{valid_lens}\n{valid_lens.shape}')
        else:
            valid_lens = valid_lens.reshape(-1)
        # 最后一轴上被掩蔽的元素使用一个非常大的负值替换，从而其softmax输出为0
        # valid_lens无论dim()几维处理后都被展平了
        X = d2l.sequence_mask(X.reshape(-1, shape[-1]), valid_lens,
                              value=-1e6)
        print(f'X.shape{X.shape}')
        return nn.functional.softmax(X.reshape(shape), dim=-1)

In [13]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([2, 3]))

valid_lens:tensor([2, 3])
torch.Size([2])
valid_lens:tensor([2, 2, 3, 3])
torch.Size([4])
X.shapetorch.Size([4, 4])


tensor([[[0.5977, 0.4023, 0.0000, 0.0000],
         [0.5877, 0.4123, 0.0000, 0.0000]],

        [[0.3859, 0.2945, 0.3197, 0.0000],
         [0.3326, 0.4566, 0.2108, 0.0000]]])

In [14]:
masked_softmax(torch.rand(2, 2, 4), torch.tensor([[1, 3], [2, 4]]))

X.shapetorch.Size([4, 4])


tensor([[[1.0000, 0.0000, 0.0000, 0.0000],
         [0.3144, 0.3824, 0.3033, 0.0000]],

        [[0.4571, 0.5429, 0.0000, 0.0000],
         [0.1766, 0.2105, 0.2188, 0.3941]]])

In [26]:
class AdditiveAttention(nn.Module):
    """ 加性注意力 q和v长度不同 """
    def __init__(self, key_size, query_size, num_hiddens, dropout, **kwargs):
        super(AdditiveAttention, self).__init__(**kwargs)
        self.W_k = nn.Linear(key_size, num_hiddens, bias=False)
        self.W_q = nn.Linear(query_size, num_hiddens, bias=False)
        self.w_v = nn.Linear(num_hiddens, 1, bias=False)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, queries, keys, values, valid_lens):
        print(f'输入形状:queries.shape{queries.shape} keys.shape:{keys.shape}')
        queries, keys = self.W_q(queries), self.W_k(keys)
        print(f'输入形状:queries.shape{queries.shape} keys.shape:{keys.shape}')
        features = queries.unsqueeze(2) + keys.unsqueeze(1)
        print(f'广播机制之后，features.shape:{features.shape}')
        features = torch.tanh(features)
        scores = self.w_v(features).squeeze(-1)
        self.attention_weights = masked_softmax(scores, valid_lens)
        print(f'attention_weights.shape{self.attention_weights.shape}\n{self.attention_weights}')
        return torch.bmm(self.dropout(self.attention_weights), values)

In [27]:
queries, keys = torch.normal(0, 1, size=(2, 1, 20)), torch.ones((2, 10, 2))
values = torch.arange(40, dtype=torch.float32).reshape(1, 10, 4).repeat(2, 1, 1)    # shape:[2, 10, 4]  repeat(2, 1, 1) 三个值分别代表第几个维度复制几遍
valid_lens = torch.tensor([2, 6])
attention = AdditiveAttention(key_size=2, query_size=20, num_hiddens=8, dropout=.1)
attention.eval()
attention(queries, keys, values, valid_lens)

输入形状:queries.shapetorch.Size([2, 1, 20]) keys.shape:torch.Size([2, 10, 2])
输入形状:queries.shapetorch.Size([2, 1, 8]) keys.shape:torch.Size([2, 10, 8])
广播机制之后，features.shape:torch.Size([2, 1, 10, 8])
valid_lens:tensor([2, 6])
torch.Size([2])
valid_lens:tensor([2, 6])
torch.Size([2])
X.shapetorch.Size([2, 10])
attention_weights.shapetorch.Size([2, 1, 10])
tensor([[[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000]],

        [[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000,
          0.0000, 0.0000]]], grad_fn=<SoftmaxBackward0>)


tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]], grad_fn=<BmmBackward0>)

In [28]:
class DotProductAttention(nn.Module):
    def __init__(self, dropout, **kwargs):
        """ 缩放点积注意力 """
        super(DotProductAttention, self).__init__(**kwargs)
        self.dropout = nn.Dropout(dropout)

    def forward(self, queries, keys, values, valid_lens=None):
        d = queries.shape[-1]
        scores = torch.bmm(queries, keys.transpose(1, 2)) / math.sqrt(d)
        self.attention_weights = masked_softmax(scores, valid_lens)
        return torch.bmm(self.dropout(self.attention_weights), values)

In [29]:
queries = torch.normal(0, 1, (2, 1, 2))
attention = DotProductAttention(dropout=.5)
attention.eval()
attention(queries, keys, values, valid_lens)

valid_lens:tensor([2, 6])
torch.Size([2])
valid_lens:tensor([2, 6])
torch.Size([2])
X.shapetorch.Size([2, 10])


tensor([[[ 2.0000,  3.0000,  4.0000,  5.0000]],

        [[10.0000, 11.0000, 12.0000, 13.0000]]])